In [2]:
#メモリのリセット
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [1]:
#ライブラリのimport
import pandas as pd
import numpy as np
%matplotlib inline
#from matplotlib import pyplot as pyp
import matplotlib.pyplot as plt

In [59]:
# Try reading the file with 'Shift_JIS' encoding
df_shiftjis_1 = pd.read_csv('20231107使用_在庫推移9月.csv', encoding='Shift_JIS')
# Load the new CSV file into a DataFrame with 'Shift_JIS' encoding
df_shiftjis_2 = pd.read_csv('在庫推移8月4週目.csv', encoding='Shift_JIS')
# Combine the old and new data
df_shiftjis = pd.concat([df_shiftjis_1, df_shiftjis_2], ignore_index=True)

# 集計したデータをCSVファイルに保存
with open('20231107使用_在庫推移9月（8月4週目含む）.csv', mode='w',newline='', encoding='shift_jis',errors='ignore') as f:
    df_shiftjis.to_csv(f)

In [62]:
#ファイル名の決定
file_path_LT = '20231117使用_所在管理LT_回収日8月第4週9月.csv'
file_path_tehaisuu = '20231107使用__202309_手配数_1Y_12次.csv'
file_path_tehaiunyou = '20231107使用_XR10_手配運用情報2300911_1Y（回収引当適用）.csv'
file_path_zaiko = '20231107使用_在庫推移9月（8月4週目含む）.csv'
target_month = 9

In [21]:
#データ準備
#元データの作り方
#所在管理MBからデータを吸い出す。'shift_jis'
#吸い出したデータを回収日をフィルタリングして新しいデータを作成する
pre_zaikodata = pd.read_csv(file_path_LT,encoding='shift_jis')
pre_zaikodata.head(5)

,かんばんシリアル,伝票番号,拠点所番地,品番,品名,収容数,仕入先名,仕入先工場名,ステータス,納入日,...,回収日時,発注取消日時,発注〜印刷LT,発注〜検収LT,発注〜順立装置入庫LT,発注〜順立装置出庫LT,発注〜組立LT,発注〜回収LT,更新日時,長期滞留フラグ
0,1Z23J00091508,XCGP130,20488,G1144ECE010,"ﾀｰﾐﾅﾙ,ﾓｰﾀｰｹｰﾌﾞﾙ",8,矢崎総業（株）,< NULL >,回収済,2023/9/29,...,2023/9/30 7:13,< NULL >,0.14,2.31,2.59,2.75,< NULL >,2.86,2023/9/30 7:13,< NULL >
1,1Z23J00068574,XCGL830,20488,G1144ECE010,"ﾀｰﾐﾅﾙ,ﾓｰﾀｰｹｰﾌﾞﾙ",8,矢崎総業（株）,< NULL >,回収済,2023/9/28,...,2023/9/30 7:13,< NULL >,0.15,1.99,2.23,3.79,< NULL >,3.86,2023/9/30 7:13,< NULL >
2,1Z23J00066432,XCGL630,30357,35847ECE010,ﾁｭｰﾌﾞT/Aﾙｰﾌﾞｱﾌﾟﾗｲ,40,（株）メタルテック,< NULL >,回収済,2023/9/28,...,2023/9/30 7:13,< NULL >,0.08,2.02,2.15,3.86,< NULL >,4.02,2023/9/30 7:13,< NULL >
3,1Z23J00065428,XCGL830,20488,G1144ECE010,"ﾀｰﾐﾅﾙ,ﾓｰﾀｰｹｰﾌﾞﾙ",8,矢崎総業（株）,< NULL >,回収済,2023/9/28,...,2023/9/30 7:13,< NULL >,0.15,1.99,2.23,3.76,< NULL >,3.86,2023/9/30 7:13,< NULL >
4,1Z23J00063618,XCGJ670,30358,35847ECE020,ﾁｭｰﾌﾞT/Aﾙｰﾌﾞｱﾌﾟﾗｲ,60,（株）メタルテック,< NULL >,回収済,2023/9/27,...,2023/9/30 7:13,< NULL >,2.07,3.99,4.28,6.86,< NULL >,7,2023/9/30 7:13,< NULL >


In [32]:
#各LTの計算

long = pre_zaikodata.iloc[:,1]
count = 0
for i in range(len(long)):
    if i % 5000==0:
            print(i)
    if ((pre_zaikodata.loc[i, '発注〜印刷LT'])!='< NULL >') and ((pre_zaikodata.loc[i, '発注〜検収LT'])!='< NULL >') and ((pre_zaikodata.loc[i, '発注〜順立装置入庫LT'])!='< NULL >')and ((pre_zaikodata.loc[i, '発注〜順立装置出庫LT'])!='< NULL >')and ((pre_zaikodata.loc[i, '発注〜回収LT'])!='< NULL >'):
        LT2 = float(pre_zaikodata.loc[i, '発注〜検収LT']) - float(pre_zaikodata.loc[i, '発注〜印刷LT'])
        LT3 = float(pre_zaikodata.loc[i, '発注〜順立装置入庫LT'])-float(pre_zaikodata.loc[i, '発注〜検収LT'])
        LT4 = float(pre_zaikodata.loc[i, '発注〜順立装置出庫LT'])-float(pre_zaikodata.loc[i, '発注〜順立装置入庫LT'])
        LT5 = float(pre_zaikodata.loc[i, '発注〜回収LT'])-float(pre_zaikodata.loc[i, '発注〜順立装置出庫LT'])
        LT6 = float(pre_zaikodata.loc[i, '発注〜回収LT'])-float(pre_zaikodata.loc[i, '発注〜検収LT'])
        pre_zaikodata.loc[i, '印刷〜検収LT'] = LT2
        pre_zaikodata.loc[i, '検収〜入庫LT'] = LT3
        pre_zaikodata.loc[i, '入庫〜出庫LT'] = LT4
        pre_zaikodata.loc[i, '出庫〜回収LT'] = LT5
        pre_zaikodata.loc[i, '社内LT（検収〜回収LT）'] = LT6
        count = count + 1
print(count)
pre_zaikodata.head(5)

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
55263


,かんばんシリアル,伝票番号,拠点所番地,品番,品名,収容数,仕入先名,仕入先工場名,ステータス,納入日,...,設計かんばん数,手配区分,基準在庫日数（基準在庫枚数込み）,在庫状態,Year,Week_Number,印刷〜検収LT,検収〜入庫LT,入庫〜出庫LT,出庫〜回収LT
0,1Z23J00091508,XCGP130,20488,G1144ECE010,"ﾀｰﾐﾅﾙ,ﾓｰﾀｰｹｰﾌﾞﾙ",8,矢崎総業（株）,< NULL >,回収済,2023-09-29,...,2.0,手配,0.611764,正常：設計値の0.75〜1倍,2023,39,2.17,0.28,0.16,0.11
1,1Z23J00068574,XCGL830,20488,G1144ECE010,"ﾀｰﾐﾅﾙ,ﾓｰﾀｰｹｰﾌﾞﾙ",8,矢崎総業（株）,< NULL >,回収済,2023-09-28,...,2.0,手配,0.611764,異常：設計値の2倍以上,2023,39,1.84,0.24,1.56,0.07
2,1Z23J00066432,XCGL630,30357,35847ECE010,ﾁｭｰﾌﾞT/Aﾙｰﾌﾞｱﾌﾟﾗｲ,40,（株）メタルテック,< NULL >,回収済,2023-09-28,...,1.0,手配,0.579410,異常：設計値の2倍以上,2023,39,1.94,0.13,1.71,0.16
3,1Z23J00065428,XCGL830,20488,G1144ECE010,"ﾀｰﾐﾅﾙ,ﾓｰﾀｰｹｰﾌﾞﾙ",8,矢崎総業（株）,< NULL >,回収済,2023-09-28,...,2.0,手配,0.611764,異常：設計値の2倍以上,2023,39,1.84,0.24,1.53,0.10
4,1Z23J00063618,XCGJ670,30358,35847ECE020,ﾁｭｰﾌﾞT/Aﾙｰﾌﾞｱﾌﾟﾗｲ,60,（株）メタルテック,< NULL >,回収済,2023-09-27,...,1.0,手配,0.609115,異常：設計値の2倍以上,2023,39,1.92,0.29,2.58,0.14


In [23]:
#基準在庫枚数を日数に変換する
#当月必要数でいい？

#datetime型に変換する
pre_zaikodata['回収日時'] = pd.to_datetime(pre_zaikodata['回収日時'], errors='coerce')
pre_zaikodata['納入日'] = pd.to_datetime(pre_zaikodata['納入日'], errors='coerce')

count = 0
tehaidata = pd.read_csv(file_path_tehaisuu,encoding='cp932')
long2 = tehaidata.iloc[:,1]

for i in range(len(long)):
    #print(pre_zaikodata.loc[i, '回収日時'],pre_zaikodata.loc[i,'品番'] )
    if pre_zaikodata.loc[i, '納入日'].month == target_month:
        #print(pre_zaikodata.loc[i, '納入日（月）'].month)
        if ((pre_zaikodata.loc[i, '発注〜印刷LT'])!='< NULL >') and ((pre_zaikodata.loc[i, '発注〜検収LT'])!='< NULL >') and ((pre_zaikodata.loc[i, '発注〜順立装置入庫LT'])!='< NULL >')and ((pre_zaikodata.loc[i, '発注〜順立装置出庫LT'])!='< NULL >')and ((pre_zaikodata.loc[i, '発注〜回収LT'])!='< NULL >'):
            hinban = pre_zaikodata.loc[i,'品番'] 
            hinban2 = hinban.replace("-", "") 
            hinban3 = hinban2.replace(" ", "") 
            if i % 5000==0:
                print(i)
            for j in range(len(long2)):
                #print(j)
                tehaihinban = tehaidata.loc[j,'品番'].replace('-', '')
                tehaihinban2 = tehaihinban.replace(' ', '')
                if hinban3 == tehaihinban2:
                    day = pre_zaikodata.loc[i,'回収日時'].day
                    index = tehaidata.columns.get_loc("翌々々月稼働日数") + day
                    pre_zaikodata.loc[i,'日量数'] = int(str(tehaidata.iloc[j,index]).replace(',', ''))
                    pre_zaikodata.loc[i,'日量数（箱数）'] = pre_zaikodata.loc[i,'日量数'] /pre_zaikodata.loc[i, '収容数']
                    if int(str(tehaidata.loc[j,'当月必要数']).replace(',', '')) != 0:
                        avenichiryo = int(str(tehaidata.loc[j,'当月必要数']).replace(',', ''))/int(tehaidata.loc[j,'当月稼働日数'])
                        pre_zaikodata.loc[i,'月平均日量数'] = avenichiryo
                        pre_zaikodata.loc[i,'月平均日量数（箱数）'] = avenichiryo//pre_zaikodata.loc[i, '収容数']
                        pre_zaikodata.loc[i,'基準在庫枚数（日数）'] = (int(tehaidata.loc[j,'基準在庫枚数'])*pre_zaikodata.loc[i, '収容数'])/avenichiryo
                    else:
                        pre_zaikodata.loc[i,'月平均日量数'] = 0
                        pre_zaikodata.loc[i,'月平均日量数（箱数）'] = 0
                        pre_zaikodata.loc[i,'基準在庫枚数（日数）'] = 0
    else:
        pre_zaikodata.loc[i,'月平均日量数'] = 0
        pre_zaikodata.loc[i,'月平均日量数（箱数）'] = 0
        pre_zaikodata.loc[i,'基準在庫枚数（日数）'] = 0
print(count)
pre_zaikodata.head(5)

0
5000
15000
25000
30000
35000
40000
0


,かんばんシリアル,伝票番号,拠点所番地,品番,品名,収容数,仕入先名,仕入先工場名,ステータス,納入日,...,印刷～検収LT,検収～入庫LT,入庫～出庫LT,出庫～回収LT,社内LT（検収〜回収LT）,日量数,日量数（箱数）,月平均日量数,月平均日量数（箱数）,基準在庫枚数（日数）
0,1Z23J00091508,XCGP130,20488,G1144ECE010,"ﾀｰﾐﾅﾙ,ﾓｰﾀｰｹｰﾌﾞﾙ",8,矢崎総業（株）,< NULL >,回収済,2023-09-29,...,2.17,0.28,0.16,0.11,0.55,0.0,0.0,503.714286,62.0,0.031764
1,1Z23J00068574,XCGL830,20488,G1144ECE010,"ﾀｰﾐﾅﾙ,ﾓｰﾀｰｹｰﾌﾞﾙ",8,矢崎総業（株）,< NULL >,回収済,2023-09-28,...,1.84,0.24,1.56,0.07,1.87,0.0,0.0,503.714286,62.0,0.031764
2,1Z23J00066432,XCGL630,30357,35847ECE010,ﾁｭｰﾌﾞT/Aﾙｰﾌﾞｱﾌﾟﾗｲ,40,（株）メタルテック,< NULL >,回収済,2023-09-28,...,1.94,0.13,1.71,0.16,2.00,0.0,0.0,503.714286,12.0,0.079410
3,1Z23J00065428,XCGL830,20488,G1144ECE010,"ﾀｰﾐﾅﾙ,ﾓｰﾀｰｹｰﾌﾞﾙ",8,矢崎総業（株）,< NULL >,回収済,2023-09-28,...,1.84,0.24,1.53,0.10,1.87,0.0,0.0,503.714286,62.0,0.031764
4,1Z23J00063618,XCGJ670,30358,35847ECE020,ﾁｭｰﾌﾞT/Aﾙｰﾌﾞｱﾌﾟﾗｲ,60,（株）メタルテック,< NULL >,回収済,2023-09-27,...,1.92,0.29,2.58,0.14,3.01,0.0,0.0,503.714286,8.0,0.119115


In [24]:
count = 0
tehaidata = pd.read_csv(file_path_tehaiunyou,encoding='cp932')
long2 = tehaidata.iloc[:,1]

for i in range(len(long)):
    if (not pd.isnull(pre_zaikodata.loc[i, '発注〜印刷LT'])) and (not pd.isnull(pre_zaikodata.loc[i, '発注〜検収LT'])) and (not pd.isnull(pre_zaikodata.loc[i, '発注〜順立装置入庫LT']))and (not pd.isnull(pre_zaikodata.loc[i, '発注〜順立装置出庫LT']))and (not pd.isnull(pre_zaikodata.loc[i, '発注〜回収LT'])):
        hinban = pre_zaikodata.loc[i,'品番'] 
        hinban2 = hinban.replace("-", "") 
        hinban3 = hinban2.replace(" ", "") 
        if i % 5000==0:
            print(i,hinban2)
        for j in range(len(long2)):
            tehaihinban = tehaidata.loc[j,'品番'].replace('-', '')
            tehaihinban2 = tehaihinban.replace(' ', '')
            #print(tehaihinban)
            if hinban3 == tehaihinban2:
                pre_zaikodata.loc[i, 'A'] = tehaidata.loc[j,'納入ｻｲｸﾙ間隔']
                pre_zaikodata.loc[i, 'B'] = tehaidata.loc[j,'納入ｻｲｸﾙ回数']
                pre_zaikodata.loc[i, 'C'] = tehaidata.loc[j,'納入ｻｲｸﾙ情報']
                pre_zaikodata.loc[i, '便Ave'] = pre_zaikodata.loc[i,'日量数（箱数）']/pre_zaikodata.loc[i, 'B']
                pre_zaikodata.loc[i, '整備室'] = tehaidata.loc[j,'整備室']
                pre_zaikodata.loc[i, '箱種類'] = tehaidata.loc[j,'登録箱種']
                pre_zaikodata.loc[i, '箱サイズ'] = tehaidata.loc[j,'容積(m3)']
                pre_zaikodata.loc[i, '箱重量'] = tehaidata.loc[j,'総重量(Kg)']
                pre_zaikodata.loc[i, '基準在庫日数'] = tehaidata.loc[j,'基準在庫日数']
                pre_zaikodata.loc[i, '基準在庫枚数'] = tehaidata.loc[j,'基準在庫枚数']
                pre_zaikodata.loc[i, '設計かんばん数'] = pre_zaikodata.loc[i,'日量数（箱数）'] + pre_zaikodata.loc[i, '基準在庫枚数']
                pre_zaikodata.loc[i, '手配区分'] = tehaidata.loc[j,'手配区分']
                MAX= tehaidata.loc[j,'基準在庫日数'] + float(pre_zaikodata.loc[i,'基準在庫枚数（日数）'])
                pre_zaikodata.loc[i,'基準在庫日数（基準在庫枚数込み）'] = MAX
                zisseki = pre_zaikodata.loc[i, '社内LT（検収〜回収LT）'] 
                if  (zisseki > MAX*0)and(zisseki < 0.25*MAX):
                    pre_zaikodata.loc[i, '在庫状態'] = '異常？：設計値の0〜0.25倍'
                elif  (zisseki > MAX*0.25)and(zisseki < 0.5*MAX):
                    pre_zaikodata.loc[i, '在庫状態'] = '異常？：設計値の0.25〜0.5倍'
                elif  (zisseki > MAX*0.5)and(zisseki < 0.75*MAX):
                    pre_zaikodata.loc[i, '在庫状態'] = '正常よりの異常？：設計値の0.5〜0.75倍'
                elif  (zisseki > MAX*0.75)and(zisseki < MAX):
                    pre_zaikodata.loc[i, '在庫状態'] = '正常：設計値の0.75〜1倍'
                elif (zisseki >= MAX)and(zisseki < MAX*1.25):
                    pre_zaikodata.loc[i, '在庫状態'] = '正常：設計値の1〜1.25倍'
                elif (zisseki >= MAX*1.25)and(zisseki < 1.5*MAX):
                    pre_zaikodata.loc[i, '在庫状態'] = '正常よりの異常：設計値の1.25〜1.5倍'
                elif (zisseki >= MAX*1.5)and(zisseki < 2*MAX):
                    pre_zaikodata.loc[i, '在庫状態'] = '異常：設計値の1.5〜2倍'
                elif (zisseki >= MAX*2):
                    pre_zaikodata.loc[i, '在庫状態'] = '異常：設計値の2倍以上'
        #print(LT2,LT3,LT4,LT5)
        count = count + 1
print(count)
pre_zaikodata.head(5)

0 G1144ECE010    
5000 41310ECB010    
10000 G9210ECE020    
15000 33490ECB010    
20000 9056439A006    
25000 9036324A005    
30000 G1174ECB010    
35000 82821CWA030    
40000 35198ECE010    
45000 35165ECE010    
50000 3559850A010    
55000 G9210ECE020    
60000 G9210ECE020    
61603


,かんばんシリアル,伝票番号,拠点所番地,品番,品名,収容数,仕入先名,仕入先工場名,ステータス,納入日,...,整備室,箱種類,箱サイズ,箱重量,基準在庫日数,基準在庫枚数,設計かんばん数,手配区分,基準在庫日数（基準在庫枚数込み）,在庫状態
0,1Z23J00091508,XCGP130,20488,G1144ECE010,"ﾀｰﾐﾅﾙ,ﾓｰﾀｰｹｰﾌﾞﾙ",8,矢崎総業（株）,< NULL >,回収済,2023-09-29,...,1Y,ＴＰ３３１フタアリ,0.014,4.71,0.58,2.0,2.0,手配,0.611764,正常：設計値の0.75〜1倍
1,1Z23J00068574,XCGL830,20488,G1144ECE010,"ﾀｰﾐﾅﾙ,ﾓｰﾀｰｹｰﾌﾞﾙ",8,矢崎総業（株）,< NULL >,回収済,2023-09-28,...,1Y,ＴＰ３３１フタアリ,0.014,4.71,0.58,2.0,2.0,手配,0.611764,異常：設計値の2倍以上
2,1Z23J00066432,XCGL630,30357,35847ECE010,ﾁｭｰﾌﾞT/Aﾙｰﾌﾞｱﾌﾟﾗｲ,40,（株）メタルテック,< NULL >,回収済,2023-09-28,...,1Y,ＴＰ３３２フタナシ,0.022,3.30,0.50,1.0,1.0,手配,0.579410,異常：設計値の2倍以上
3,1Z23J00065428,XCGL830,20488,G1144ECE010,"ﾀｰﾐﾅﾙ,ﾓｰﾀｰｹｰﾌﾞﾙ",8,矢崎総業（株）,< NULL >,回収済,2023-09-28,...,1Y,ＴＰ３３１フタアリ,0.014,4.71,0.58,2.0,2.0,手配,0.611764,異常：設計値の2倍以上
4,1Z23J00063618,XCGJ670,30358,35847ECE020,ﾁｭｰﾌﾞT/Aﾙｰﾌﾞｱﾌﾟﾗｲ,60,（株）メタルテック,< NULL >,回収済,2023-09-27,...,1Y,ＴＰ３３２フタナシ,0.022,3.50,0.49,1.0,1.0,手配,0.609115,異常：設計値の2倍以上


In [34]:
# 集計したデータをCSVファイルに保存
with open('kanbandata.csv', mode='w',newline='', encoding='shift_jis',errors='ignore') as f:
    pre_zaikodata.to_csv(f)
#
traindata = pd.read_csv('kanbandata.csv',encoding='cp932')
traindata.head(5)

c:\users\1082794-z100\.pyenv\pyenv-win\versions\3.7.9\lib\site-packages\IPython\core\interactiveshell.py:3553: DtypeWarning: Columns (11,25) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,かんばんシリアル,伝票番号,拠点所番地,品番,品名,収容数,仕入先名,仕入先工場名,ステータス,...,設計かんばん数,手配区分,基準在庫日数（基準在庫枚数込み）,在庫状態,Year,Week_Number,印刷～検収LT,検収～入庫LT,入庫～出庫LT,出庫～回収LT
0,0,1Z23J00091508,XCGP130,20488,G1144ECE010,"ﾀｰﾐﾅﾙ,ﾓｰﾀｰｹｰﾌﾞﾙ",8,矢崎総業（株）,< NULL >,回収済,...,2.0,手配,0.611764,正常：設計値の0.75～1倍,2023,39,2.17,0.28,0.16,0.11
1,1,1Z23J00068574,XCGL830,20488,G1144ECE010,"ﾀｰﾐﾅﾙ,ﾓｰﾀｰｹｰﾌﾞﾙ",8,矢崎総業（株）,< NULL >,回収済,...,2.0,手配,0.611764,異常：設計値の2倍以上,2023,39,1.84,0.24,1.56,0.07
2,2,1Z23J00066432,XCGL630,30357,35847ECE010,ﾁｭｰﾌﾞT/Aﾙｰﾌﾞｱﾌﾟﾗｲ,40,（株）メタルテック,< NULL >,回収済,...,1.0,手配,0.579410,異常：設計値の2倍以上,2023,39,1.94,0.13,1.71,0.16
3,3,1Z23J00065428,XCGL830,20488,G1144ECE010,"ﾀｰﾐﾅﾙ,ﾓｰﾀｰｹｰﾌﾞﾙ",8,矢崎総業（株）,< NULL >,回収済,...,2.0,手配,0.611764,異常：設計値の2倍以上,2023,39,1.84,0.24,1.53,0.10
4,4,1Z23J00063618,XCGJ670,30358,35847ECE020,ﾁｭｰﾌﾞT/Aﾙｰﾌﾞｱﾌﾟﾗｲ,60,（株）メタルテック,< NULL >,回収済,...,1.0,手配,0.609115,異常：設計値の2倍以上,2023,39,1.92,0.29,2.58,0.14


In [63]:
# 'cp932'エンコーディングを使用してCSVファイルを読み込みます。
new_kanban_data = pd.read_csv(file_path_zaiko, encoding='cp932')

# '計測日時'を日時型に変換し、週番号を含む新しい列を追加します。
new_kanban_data['計測日時'] = pd.to_datetime(new_kanban_data['計測日時'], errors='coerce')
new_kanban_data.dropna(subset=['計測日時'], inplace=True)
new_kanban_data['Year'] = new_kanban_data['計測日時'].dt.year
new_kanban_data['Week_Number'] = new_kanban_data['計測日時'].dt.week

# '品番'と'Week_Number'でグループ化し、'在庫数（箱）', '入庫数（箱）', '出庫数（箱）'を合計します。
weekly_inventory_aggregated = new_kanban_data.groupby(['品番', 'Year', 'Week_Number']).agg({
    '在庫数（箱）': 'median',
    '入庫数（箱）': 'median',
    '出庫数（箱）': 'median'
}).reset_index()

# 集計したデータを新しいCSVファイルに保存します。
with open('kari1.csv', mode='w',newline='', encoding='shift_jis',errors='ignore') as f:
    weekly_inventory_aggregated.to_csv(f)

weekly_inventory_aggregated.head(300)

c:\users\1082794-z100\.pyenv\pyenv-win\versions\3.7.9\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  


,品番,Year,Week_Number,在庫数（箱）,入庫数（箱）,出庫数（箱）
0,019120LC030,2023,34,10.0,0.0,0.0
1,019120LC030,2023,35,11.0,0.0,0.0
2,019120LC030,2023,36,12.0,0.0,0.0
3,019120LC030,2023,37,11.5,0.0,0.0
4,019120LC030,2023,38,11.0,0.0,0.0
...,...,...,...,...,...,...
295,35145ECE020,2023,37,5.0,0.0,0.0
296,35145ECE020,2023,38,5.0,0.0,0.0
297,35145ECE020,2023,39,7.0,0.0,0.0
298,35165ECE010,2023,34,7.0,0.0,0.0


In [64]:
# '納入日'を日時型に変換
pre_zaikodata['回収日時'] = pd.to_datetime(pre_zaikodata['回収日時'], errors='coerce')
# 日時型に変換できなかった行を削除
pre_zaikodata.dropna(subset=['回収日時'], inplace=True)

pre_zaikodata.dropna(subset=['印刷～検収LT'], inplace=True)
pre_zaikodata.dropna(subset=['検収～入庫LT'], inplace=True)
pre_zaikodata.dropna(subset=['入庫～出庫LT'], inplace=True)
pre_zaikodata.dropna(subset=['出庫～回収LT'], inplace=True)
#pre_zaikodata.dropna(subset=['社内LT（検収～回収LT）'], inplace=True)

# 'Week_Number' 列を追加（年と週の番号を取得）
pre_zaikodata['Year'] = pre_zaikodata['回収日時'].dt.year
pre_zaikodata['Week_Number'] = pre_zaikodata['回収日時'].dt.week

# '品番'と'Week_Number'でグループ化し、'収容数'などを合計
weekly_aggregated = pre_zaikodata.groupby(['品番', '仕入先名','Year', 'Week_Number', '基準在庫日数', '基準在庫枚数','箱種類','A','B','C']).agg({'収容数': 'median', '納入便': 'median', '印刷〜検収LT': 'median', '検収～入庫LT': 'median','入庫～出庫LT': 'median','出庫～回収LT': 'median','社内LT（検収〜回収LT）': 'median','日量数（箱数）':'median'})
# '品番'と'Week_Number'でグループ化し、各グループの行数（かんばん数）をカウント
weekly_kanban_count = pre_zaikodata.groupby(['品番','仕入先名','Year', 'Week_Number', '基準在庫日数', '基準在庫枚数','箱種類','A','B','C']).size().reset_index(name='回収かんばん数')

# 2つの集計データセットを結合
merged_weekly_data = pd.merge(weekly_aggregated, weekly_kanban_count, on=['品番', 'Year', 'Week_Number', '基準在庫日数', '基準在庫枚数','箱種類','A','B','C'], how='outer')

# 集計したデータをCSVファイルに保存
with open('kari2.csv', mode='w',newline='', encoding='shift_jis',errors='ignore') as f:
    merged_weekly_data.to_csv(f)

merged_weekly_data.head(30)

c:\users\1082794-z100\.pyenv\pyenv-win\versions\3.7.9\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  


,品番,Year,Week_Number,基準在庫日数,基準在庫枚数,箱種類,A,B,C,収容数,納入便,印刷〜検収LT,検収～入庫LT,入庫～出庫LT,出庫～回収LT,社内LT（検収〜回収LT）,日量数（箱数）,仕入先名,回収かんばん数
0,019128GA010,2023,35,1.07,1.0,ＴＰ３４２フタナシ,1.0,2.0,3.64,50.0,1.0,1.850,0.560,0.070,0.210,0.840,0.000000,石黒ゴム工業（株）,1
1,019128GA010,2023,36,1.07,1.0,ＴＰ３４２フタナシ,1.0,2.0,3.64,50.0,1.0,1.850,0.310,1.335,0.230,2.130,18.900000,石黒ゴム工業（株）,62
2,019128GA010,2023,37,1.07,1.0,ＴＰ３４２フタナシ,1.0,2.0,3.64,50.0,1.5,1.850,0.280,2.440,0.235,3.640,16.920000,石黒ゴム工業（株）,90
3,019128GA010,2023,38,1.07,1.0,ＴＰ３４２フタナシ,1.0,2.0,3.64,50.0,1.0,1.850,0.290,2.660,0.250,3.510,17.540000,石黒ゴム工業（株）,87
4,019128GA010,2023,39,1.07,1.0,ＴＰ３４２フタナシ,1.0,2.0,3.64,50.0,2.0,1.850,0.345,2.625,0.220,3.575,17.560000,石黒ゴム工業（株）,92
5,01912ECB010,2023,36,0.85,1.0,ＴＰ３３２フタナシ,1.0,2.0,3.64,300.0,1.0,3.270,0.300,2.590,0.300,3.230,6.300000,石黒ゴム工業（株）,22
6,01912ECB010,2023,37,0.85,1.0,ＴＰ３３２フタナシ,1.0,2.0,3.64,300.0,1.0,1.850,0.250,2.620,0.250,3.410,5.633333,石黒ゴム工業（株）,31
7,01912ECB010,2023,38,0.85,1.0,ＴＰ３３２フタナシ,1.0,2.0,3.64,300.0,1.0,1.850,0.310,2.440,0.270,3.670,5.846667,石黒ゴム工業（株）,29
8,01912ECB010,2023,39,0.85,1.0,ＴＰ３３２フタナシ,1.0,2.0,3.64,300.0,2.0,1.850,0.390,2.550,0.280,3.440,5.853333,石黒ゴム工業（株）,31
9,01912ECB040,2023,36,0.25,1.0,ＴＰ３４２フタナシ,1.0,1.0,2.12,50.0,1.0,2.180,0.010,1.830,0.220,2.100,18.900000,愛産樹脂工業（株）,77


In [65]:
# 2つの集計データセットを結合
merged_data = pd.merge(weekly_inventory_aggregated, merged_weekly_data, on=['品番', 'Year', 'Week_Number'], how='inner')
# '基準在庫日数' から '社内LT（検収～回収LT）' を引いた新しい列 'LT_差分' を追加
merged_data['社内LT/設計値LT'] = merged_data['社内LT（検収〜回収LT）']/merged_data['基準在庫日数']
#
merged_data['順立装置在庫量/設計値MIN']=merged_data['在庫数（箱）']/(0.1*(merged_data['日量数（箱数）']*merged_data['A']*(1+merged_data['C'])/merged_data['B']))

# Load the new CSV file into a DataFrame with 'Shift_JIS' encoding
pitch = pd.read_csv('不等ピッチ情報.csv', encoding='Shift_JIS')

longi = len(merged_data.loc[:,"品番"])
longj = len(pitch.loc[:,"仕入先名"])

for i in range(longi):
    for j in range(longj):
        if (merged_data.loc[i, '仕入先名'] == pitch.loc[j,'仕入先名']):
            merged_data.loc[i,"不等ピッチ係数"] = pitch.loc[j,"不等ピッチ係数日"]
            
merged_data['順立装置在庫量/設計値MAX'] = merged_data['在庫数（箱）']/(merged_data['順立装置在庫量/設計値MIN'] + merged_data['日量数（箱数）']/merged_data['B'] + merged_data['日量数（箱数）']*merged_data.loc[i,"不等ピッチ係数"])

#S = int(str(df_design.loc[j,'収容数']).replace(',', ''))
#df_merge.loc[count,'最小値'] = 0.1 * ((daily/S)  * A * (1 + C) / B) 
#df_merge.loc[count,'最大値'] = df_merge.loc[count,'最小値'] + (daily / S)/B + (daily/S)*df_design.loc[j,'不等ピッチ']

# 集計したデータをCSVファイルに保存
with open('kari5.csv', mode='w',newline='', encoding='shift_jis',errors='ignore') as f:
    merged_data.to_csv(f)

merged_data.head(300)

,品番,Year,Week_Number,在庫数（箱）,入庫数（箱）,出庫数（箱）,基準在庫日数,基準在庫枚数,箱種類,A,...,入庫～出庫LT,出庫～回収LT,社内LT（検収〜回収LT）,日量数（箱数）,仕入先名,回収かんばん数,社内LT/設計値LT,順立装置在庫量/設計値MIN,不等ピッチ係数,順立装置在庫量/設計値MAX
0,019128GA010,2023,35,49.0,0.0,1.0,1.07,1.0,ＴＰ３４２フタナシ,1.0,...,0.070,0.210,0.840,0.00,石黒ゴム工業（株）,1,0.785047,inf,0.29,0.000000
1,019128GA010,2023,36,41.0,0.0,1.0,1.07,1.0,ＴＰ３４２フタナシ,1.0,...,1.335,0.230,2.130,18.90,石黒ゴム工業（株）,62,1.990654,9.350483,0.29,2.076424
2,019128GA010,2023,37,40.0,0.0,1.0,1.07,1.0,ＴＰ３４２フタナシ,1.0,...,2.440,0.235,3.640,16.92,石黒ゴム工業（株）,90,3.401869,10.189940,0.29,2.051709
3,019128GA010,2023,38,37.0,0.0,1.0,1.07,1.0,ＴＰ３４２フタナシ,1.0,...,2.660,0.250,3.510,17.54,石黒ゴム工業（株）,87,3.280374,9.092518,0.29,1.974437
4,019128GA010,2023,39,40.0,0.0,1.0,1.07,1.0,ＴＰ３４２フタナシ,1.0,...,2.625,0.220,3.575,17.56,石黒ゴム工業（株）,92,3.341121,9.818553,0.29,2.053751
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,9010506A089,2023,38,3.0,0.0,0.0,0.50,3.0,ＴＰ３３１フタナシ,1.0,...,4.180,0.935,5.495,1.17,（株）青山製作所,6,10.990000,22.758307,0.08,0.130368
296,9010506A089,2023,39,3.0,0.0,0.0,0.50,3.0,ＴＰ３３１フタナシ,1.0,...,2.205,0.890,3.955,1.21,（株）青山製作所,6,7.910000,22.005966,0.08,0.134722
297,9010508A014,2023,35,19.0,0.0,1.0,0.54,3.0,ＴＰ３３１フタナシ,1.0,...,0.410,0.120,0.920,0.00,（株）青山製作所,8,1.703704,inf,0.08,0.000000
298,9010508A014,2023,36,23.0,0.0,1.0,0.54,3.0,ＴＰ３３１フタナシ,1.0,...,0.540,0.160,0.940,71.49,（株）青山製作所,346,1.740741,2.855532,0.08,1.253745


In [72]:
# 新しいCSVファイルのパスを定義します。
#shiftjisでMBから落とす
new_file_path = '組立稼働率（実績）.csv'

# 'cp932'エンコーディングを使用してCSVファイルを読み込みます。
AVAILABLE_RATE = pd.read_csv(new_file_path, encoding='cp932')

# データフレームのインデックスを日付型に変換
AVAILABLE_RATE['LINE_DATE'] = pd.to_datetime(AVAILABLE_RATE['LINE_DATE'])
#AVAILABLE_RATE.set_index('LINE_DATE', inplace=True)
# 'WEEK_NUMBER'列を追加し、'LINE_DATE'の週番号を計算
AVAILABLE_RATE['Week_Number'] = AVAILABLE_RATE['LINE_DATE'].dt.week
AVAILABLE_RATE['WEEKDAY'] = AVAILABLE_RATE['LINE_DATE'].dt.weekday

# 土日を除外
AVAILABLE_RATE2 = AVAILABLE_RATE[(AVAILABLE_RATE['WEEKDAY'] != 5) & (AVAILABLE_RATE['WEEKDAY'] != 6)]

# 'AVAILABLE_RATE'列を週単位でリサンプリングし、平均を計算
weekly_average_available_rate = AVAILABLE_RATE2.groupby('Week_Number').agg({'OPERATION_RATE': 'median', 'KAKO_CNT': 'median'})

weekly_average_available_rate = weekly_average_available_rate.rename(columns={'OPERATION_RATE': '時間稼働率'})
weekly_average_available_rate = weekly_average_available_rate.rename(columns={'KAKO_CNT': '加工数'})

# 集計したデータを新しいCSVファイルに保存します。
with open('kari4.csv', mode='w',newline='', encoding='shift_jis',errors='ignore') as f:
    weekly_average_available_rate.to_csv(f)

#AVAILABLE_RATE.head(300)
weekly_average_available_rate.head(30)

c:\users\1082794-z100\.pyenv\pyenv-win\versions\3.7.9\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  if sys.path[0] == "":


,時間稼働率,加工数
Week_Number,,
14,97.145,409.5
15,98.095,463.0
16,95.640,449.5
17,93.025,490.5
18,0.000,0.0
19,79.925,350.0
20,94.195,506.5
21,91.335,496.0
22,90.475,506.5


In [73]:
# 2つの集計データセットを結合
merged_data = pd.merge(merged_data, weekly_average_available_rate, on=['Week_Number'], how='inner')
merged_data.dropna(subset=['日量数（箱数）'], inplace=True)
# 集計したデータを新しいCSVファイルに保存します。
with open('kari6.csv', mode='w',newline='', encoding='shift_jis',errors='ignore') as f:
    merged_data.to_csv(f)
merged_data.head(300)

,品番,Year,Week_Number,在庫数（箱）,入庫数（箱）,出庫数（箱）,基準在庫日数,基準在庫枚数,箱種類,A,...,社内LT（検収〜回収LT）,日量数（箱数）,仕入先名,回収かんばん数,社内LT/設計値LT,順立装置在庫量/設計値MIN,不等ピッチ係数,順立装置在庫量/設計値MAX,時間稼働率,加工数
0,019128GA010,2023,35,49.0,0.0,1.0,1.07,1.0,ＴＰ３４２フタナシ,1.0,...,0.84,0.000,石黒ゴム工業（株）,1,0.785047,inf,0.29,0.000000,99.885,475.0
1,3040052001B,2023,35,2.0,0.0,0.0,0.52,3.0,ＴＰ３３１フタナシ,1.0,...,0.94,0.000,（株）青山製作所,2,1.807692,inf,0.08,0.000000,99.885,475.0
2,33490ECB010,2023,35,68.0,0.0,4.0,0.82,3.0,ＴＰ３３１．５フタアリ,1.0,...,0.89,0.000,マレリ（株）,23,1.085366,inf,0.23,0.000000,99.885,475.0
3,33490ECE010,2023,35,109.0,0.0,6.0,0.81,3.0,ＴＰ３３１．５フタアリ,1.0,...,0.92,0.000,マレリ（株）,19,1.135802,inf,0.23,0.000000,99.885,475.0
4,35174ECB010,2023,35,18.0,0.0,1.0,0.87,1.0,ＴＰ３３２フタナシ,1.0,...,0.76,0.000,（株）浅賀井製作所,1,0.873563,inf,0.13,0.000000,99.885,475.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,9056463A035,2023,37,1.0,0.0,0.0,0.93,3.0,TP131ﾌﾀﾅｼ,1.0,...,3.88,0.000,（株）ムロコーポレーション,3,4.172043,inf,0.00,0.000000,99.440,462.0
296,9056463A036,2023,37,1.0,0.0,0.0,0.93,3.0,TP131ﾌﾀﾅｼ,1.0,...,3.95,0.000,（株）ムロコーポレーション,3,4.247312,inf,0.00,0.000000,99.440,462.0
297,9094906A012,2023,37,6.0,0.0,0.0,0.59,3.0,AW131ﾌﾀﾅｼ,1.0,...,1.42,5.820,中庸スプリング（株）,35,2.406780,6.173221,0.10,0.640053,99.440,462.0
298,G1144ECB010,2023,37,33.0,0.0,2.0,0.63,2.0,ＴＰ３３１フタナシ,1.0,...,1.28,62.000,住友電気工業（株）,332,2.031746,1.622738,0.00,0.494584,99.440,462.0


In [83]:
# 品番と週番号でデータを並べ替え
merged_data2 = merged_data.sort_values(by=['品番', 'Week_Number'])
# 同じ品番ごとに在庫数の週間差分を計算
merged_data2['先週からの順立装置在庫量増加率'] = merged_data2.groupby('品番')['在庫数（箱）'].pct_change()
merged_data2['先週からの社内LT増加率'] = merged_data2.groupby('品番')['社内LT（検収〜回収LT）'].pct_change()
merged_data2['先週からの回収かんばん増加率'] = merged_data2.groupby('品番')['回収かんばん数'].pct_change()
# 差分がNaN（最初の週）である場合は0に置き換え
merged_data2['先週からの順立装置在庫量増加率'].fillna(0, inplace=True)
merged_data2['先週からの社内LT増加率'].fillna(0, inplace=True)
merged_data2['先週からの回収かんばん増加率'].fillna(0, inplace=True)
merged_data2.head(30)

,品番,Year,Week_Number,在庫数（箱）,入庫数（箱）,出庫数（箱）,基準在庫日数,基準在庫枚数,箱種類,A,...,回収かんばん数,社内LT/設計値LT,順立装置在庫量/設計値MIN,不等ピッチ係数,順立装置在庫量/設計値MAX,時間稼働率,加工数,先週からの順立装置在庫量増加率,先週からの社内LT増加率,先週からの回収かんばん増加率
0,019128GA010,2023,35,49.0,0.0,1.0,1.07,1.0,ＴＰ３４２フタナシ,1.0,...,1,0.785047,inf,0.29,0.000000,99.885,475.0,0.000000,0.000000,0.000000
36,019128GA010,2023,36,41.0,0.0,1.0,1.07,1.0,ＴＰ３４２フタナシ,1.0,...,62,1.990654,9.350483,0.29,2.076424,99.895,480.0,-0.163265,1.535714,61.000000
169,019128GA010,2023,37,40.0,0.0,1.0,1.07,1.0,ＴＰ３４２フタナシ,1.0,...,90,3.401869,10.189940,0.29,2.051709,99.440,462.0,-0.024390,0.708920,0.451613
310,019128GA010,2023,38,37.0,0.0,1.0,1.07,1.0,ＴＰ３４２フタナシ,1.0,...,87,3.280374,9.092518,0.29,1.974437,99.905,450.0,-0.075000,-0.035714,-0.033333
458,019128GA010,2023,39,40.0,0.0,1.0,1.07,1.0,ＴＰ３４２フタナシ,1.0,...,92,3.341121,9.818553,0.29,2.053751,99.955,464.0,0.081081,0.018519,0.057471
37,01912ECB010,2023,36,13.0,0.0,0.0,0.85,1.0,ＴＰ３３２フタナシ,1.0,...,22,3.800000,8.894362,0.29,1.051834,99.895,480.0,0.000000,0.000000,0.000000
170,01912ECB010,2023,37,13.0,0.0,0.0,0.85,1.0,ＴＰ３３２フタナシ,1.0,...,31,4.011765,9.946950,0.29,0.996529,99.440,462.0,0.000000,0.055728,0.409091
311,01912ECB010,2023,38,12.0,0.0,0.0,0.85,1.0,ＴＰ３３２フタナシ,1.0,...,29,4.317647,8.846774,0.29,0.994824,99.905,450.0,-0.076923,0.076246,-0.064516
459,01912ECB010,2023,39,14.0,0.0,0.0,0.85,1.0,ＴＰ３３２フタナシ,1.0,...,31,4.047059,10.309481,0.29,1.034828,99.955,464.0,0.166667,-0.062670,0.068966
38,01912ECB040,2023,36,30.5,0.0,1.0,0.25,1.0,ＴＰ３４２フタナシ,1.0,...,77,8.400000,5.172297,0.00,1.219156,99.895,480.0,0.000000,0.000000,0.000000


In [84]:
# 新しいCSVファイルのパスを定義します。
new_file_path = '20231107使用_使用工程.csv'
# 'cp932'エンコーディングを使用してCSVファイルを読み込みます。
step = pd.read_csv(new_file_path, encoding='cp932')

longi = len(merged_data2.loc[:,"品番"])
longj = len(step.loc[:,"品番"])

for i in range(longi):
    hinban = merged_data2.loc[i,'品番'] 
    hinban2 = hinban.replace("-", "") 
    hinban3 = hinban2.replace(" ", "") 
    for j in range(longj):
        stephinban = step.loc[j,'品番'].replace('-', '')
        stephinban2 = stephinban.replace(' ', '')
        if (hinban3 == stephinban2):
            merged_data2.loc[i,"使用工程"] = step.loc[j,"使用工程"]
    
merged_data2.head(5)

,品番,Year,Week_Number,在庫数（箱）,入庫数（箱）,出庫数（箱）,基準在庫日数,基準在庫枚数,箱種類,A,...,社内LT/設計値LT,順立装置在庫量/設計値MIN,不等ピッチ係数,順立装置在庫量/設計値MAX,時間稼働率,加工数,先週からの順立装置在庫量増加率,先週からの社内LT増加率,先週からの回収かんばん増加率,使用工程
0,019128GA010,2023,35,49.0,0.0,1.0,1.07,1.0,ＴＰ３４２フタナシ,1.0,...,0.785047,inf,0.29,0.000000,99.885,475.0,0.000000,0.000000,0.000000,8.ファイナル後半
36,019128GA010,2023,36,41.0,0.0,1.0,1.07,1.0,ＴＰ３４２フタナシ,1.0,...,1.990654,9.350483,0.29,2.076424,99.895,480.0,-0.163265,1.535714,61.000000,8.ファイナル後半
169,019128GA010,2023,37,40.0,0.0,1.0,1.07,1.0,ＴＰ３４２フタナシ,1.0,...,3.401869,10.189940,0.29,2.051709,99.440,462.0,-0.024390,0.708920,0.451613,8.ファイナル後半
310,019128GA010,2023,38,37.0,0.0,1.0,1.07,1.0,ＴＰ３４２フタナシ,1.0,...,3.280374,9.092518,0.29,1.974437,99.905,450.0,-0.075000,-0.035714,-0.033333,8.ファイナル後半
458,019128GA010,2023,39,40.0,0.0,1.0,1.07,1.0,ＴＰ３４２フタナシ,1.0,...,3.341121,9.818553,0.29,2.053751,99.955,464.0,0.081081,0.018519,0.057471,8.ファイナル後半


In [ ]:
#納入数追加
#入庫数合計
#出庫数合計

In [85]:
# Week_Numberが36のデータを削除
merged_data2 = merged_data2[merged_data2['Week_Number'] != 35]

# 集計したデータを新しいCSVファイルに保存します。
with open('data.csv', mode='w',newline='', encoding='shift_jis',errors='ignore') as f:
    merged_data2.to_csv(f)

# 確認のため、削除後のデータの最初の5行を表示
merged_data2.head(5)

,品番,Year,Week_Number,在庫数（箱）,入庫数（箱）,出庫数（箱）,基準在庫日数,基準在庫枚数,箱種類,A,...,社内LT/設計値LT,順立装置在庫量/設計値MIN,不等ピッチ係数,順立装置在庫量/設計値MAX,時間稼働率,加工数,先週からの順立装置在庫量増加率,先週からの社内LT増加率,先週からの回収かんばん増加率,使用工程
36,019128GA010,2023,36,41.0,0.0,1.0,1.07,1.0,ＴＰ３４２フタナシ,1.0,...,1.990654,9.350483,0.29,2.076424,99.895,480.0,-0.163265,1.535714,61.000000,8.ファイナル後半
169,019128GA010,2023,37,40.0,0.0,1.0,1.07,1.0,ＴＰ３４２フタナシ,1.0,...,3.401869,10.189940,0.29,2.051709,99.440,462.0,-0.024390,0.708920,0.451613,8.ファイナル後半
310,019128GA010,2023,38,37.0,0.0,1.0,1.07,1.0,ＴＰ３４２フタナシ,1.0,...,3.280374,9.092518,0.29,1.974437,99.905,450.0,-0.075000,-0.035714,-0.033333,8.ファイナル後半
458,019128GA010,2023,39,40.0,0.0,1.0,1.07,1.0,ＴＰ３４２フタナシ,1.0,...,3.341121,9.818553,0.29,2.053751,99.955,464.0,0.081081,0.018519,0.057471,8.ファイナル後半
37,01912ECB010,2023,36,13.0,0.0,0.0,0.85,1.0,ＴＰ３３２フタナシ,1.0,...,3.800000,8.894362,0.29,1.051834,99.895,480.0,0.000000,0.000000,0.000000,8.ファイナル後半


In [57]:
step.head(300)

,品番,使用工程
0,34989ECE030,NaN
1,9036340A010,NaN
2,35145ECE020,2.MGケース
3,33490ECB010,7.ファイナル前半
4,33490ECE010,7.ファイナル前半
...,...,...
143,9036652A004,1.デフ・大物ギヤ・Brg
144,9036628A007,1.デフ・大物ギヤ・Brg
145,9036630A017,1.デフ・大物ギヤ・Brg
146,9036628A006,1.デフ・大物ギヤ・Brg
